Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Imports

In [2]:
import random
import glob
from csv import writer
import csv
import os
import shutil
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import time
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import Image as ig
from PIL import Image as im
import pandas as pd


In [3]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
FILE_PATH_1 = '/content/gdrive/My Drive/Mobisy/trainedModels/mnist_image_classification_model.pt'
FILE_PATH_2 = '/content/gdrive/My Drive/Mobisy/trainedModels/summation_model.pt'

Model 1 - Classification Model

In [5]:
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1) 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1)                                  
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2)                               
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1)                                  
        self.fc1 = nn.Linear(8*11*11, 120)
        self.fc2 = nn.Linear(120, 10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))         # output: 26*26 -> # output: 25*25
        x = self.pool2(self.relu(self.conv2(x)))         # output:  12*12 -> 11*11
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net1 = Model1()

Model 2 - Summation of 2 Numbers

In [6]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()                             
        self.fc1 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        return x

net2 = Model2()

Load Model 1

In [7]:
net1.load_state_dict(torch.load(FILE_PATH_1))
net1.eval()

Model1(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(2, 2))
  (pool2): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=968, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
  (relu): ReLU()
)

Load Model 2


In [8]:
net2.load_state_dict(torch.load(FILE_PATH_2))
net2.eval()

Model2(
  (fc1): Linear(in_features=2, out_features=1, bias=True)
)

Test Image

In [9]:
data_path = '/content/gdrive/My Drive/Mobisy/trainingSet/'
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Grayscale(1),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Grayscale(1),
    ]),
}
image_datasets = {x: datasets.ImageFolder(root=os.path.join(data_path, x),transform=data_transforms[x]) for x in ['train', 'val']}
target = image_datasets['train'].classes

In [10]:
def testImage(model, PATH):
    image = Image.open(PATH)
    model = model.to(device)
    transform = transforms.ToTensor()
    img_tensor = transform(image)
    img_tensor = torch.unsqueeze(img_tensor, 0)
    model.to(device)
    img_tensor = img_tensor.to(device)
    img_tensor = img_tensor.float()
    pred = model(img_tensor)
    check = F.softmax(pred, dim=1)
    idx = torch.argmax(check, dim=1)
    
    return target[idx]

Test Summation

In [11]:
def testSummation(model,input):
    pred=model(input)
    return pred.item()


Load Test Images

In [12]:
def evaluate(original,predicted):
  acc=0
  for i in range(0,len(predicted)):
    if(original[i]==predicted[i]):
      acc=acc+1
  print("Test set accuracy: ",acc/len(predicted))

Final Result

In [13]:
index=0
test_result=[]
test_data=pd.read_csv('/content/gdrive/My Drive/Mobisy/test_data.csv')
for ind in test_data.index:
  image_output=int(testImage(net1,'/'+test_data['file_path'][ind][1:]))
  second_model_input=[[image_output,test_data['random_number'][ind]]]
  second_input = torch.as_tensor(second_model_input).to(device)
  second_input=second_input.float()
  summation=testSummation(net2,second_input)
  #print(test_data['file_path'][ind],image_output,second_model_input,summation)
  test_result.append(round(summation))
evaluate(test_data['sum'],test_result)


Test set accuracy:  0.9028571428571428
